In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/joestubbs/coe379L-fa25/main/datasets/unit01/project1.csv')

In [3]:
df.shape

(131165, 12)

**1.1 Shape and size:** The data set is 131,165 rows and 12 columns

In [4]:
df.dtypes

Animal ID           object
Date of Birth       object
Name                object
DateTime            object
MonthYear           object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [5]:
df.head()
for col in df.columns:
    print(f"Column: {col}")
    print(df[col].unique())
    print("-" * 100)

Column: Animal ID
['A680855' 'A680857' 'A680858' ... 'A672142' 'A675119' 'A678559']
----------------------------------------------------------------------------------------------------
Column: Date of Birth
['5/25/14' '4/22/14' '6/12/14' ... '3/9/25' '2/28/25' '3/19/25']
----------------------------------------------------------------------------------------------------
Column: Name
[nan 'Jenny Tyson' '*Stetson' ... '*Gorgeous George' '*Hen' 'Nahtha']
----------------------------------------------------------------------------------------------------
Column: DateTime
['2014-06-10T00:00:00-05:00' '2014-07-11T00:00:00-05:00'
 '2014-07-12T00:00:00-05:00' ... '2014-02-08T10:14:00'
 '2014-02-07T10:14:00' '2014-05-14T08:00:00']
----------------------------------------------------------------------------------------------------
Column: MonthYear
['Jun-14' 'Jul-14' 'Aug-14' 'Sep-14' 'Dec-14' 'Jan-15' 'Feb-15' 'Mar-15'
 'Apr-15' 'May-15' 'Jun-15' 'Jul-15' 'Aug-15' 'Sep-15' 'Oct-15' 'Nov-15'
 'D

**1.2 Data Types:** All the columns are 'object's, probably because the data isn't cleaned. That said, based on head and the unique values printing that I did: Animal ID is a string; Date of Birth is a Date string (needs to be parsed into a date from a string though); Name is  a string, DateTime is a date time string(a string that needs to be parsed, or an actual DateTime). MonthYear is a date string. OutcomeType, OutcomeSubtype, Animal Type, Breed, and Color are all categorical data that can be onehot encoded. Finally, Age upon Outcome seems to be a descriptive string of the age as a integer of days, weeks, months, or years.
I'm going to clean up all this data.

In [6]:
df = df.drop('Breed', axis=1).drop('Name', axis=1).drop('Outcome Subtype', axis=1).drop_duplicates()

In [7]:
mask = df['MonthYear'].notna() & df['DateTime'].isna()
has_missing = mask.any()
has_missing

np.False_

I wanted to make sure that there weren't any rows where MonthYear was present but DateTime was not. Now I can drop the MonthYear column since it's redundant.

In [8]:
df = df.drop('MonthYear', axis=1)

In [9]:
df = df[df['Outcome Type'].isin(['Transfer', 'Adoption'])]
df['Outcome Type'] = df['Outcome Type'].map({'Transfer': 0, 'Adoption': 1})

In [11]:
row_count = len(df)

num_unknown_sex = (df['Sex upon Outcome'] == 'Unknown').sum()
portion_us_unknown = num_unknown_sex / row_count

print(f"Portion Sex upon Outcome Unknown: {portion_us_unknown:.2%} of total")

Portion Sex upon Outcome Unknown: 3.89% of total


I'm OK swapping out 'Unknown' Sex upon Outcome values for the most frequent value (mode), but I don't want to replace Invalid Outcome Subtypes with some other value since that would have a massive impact on the data. Instead, I'll leave "NaN" as a legitimate category to onehot encode.

In [12]:
valid_dob = df['Date of Birth'].apply(lambda x: isinstance(x, str) and x.strip() != "")
valid_dt = df['DateTime'].apply(lambda x: isinstance(x, str) and x.strip() != "")
both_valid = valid_dob & valid_dt
both_valid.mean()

np.float64(1.0)

Since all of my data has valid DoBs and DateTimes, I can use them to calculate an 'Age entered shelter' in days and a 'Duration in shelter' in days
I'm going to transform, add, and remove date-related features into data I find more interesting/revealing than just arbitrary dates

In [13]:
admitted_to_shelter = df['DateTime'].astype(str).str.strip()
parsed_admitted_to_shelter = pd.to_datetime(admitted_to_shelter, format="%Y-%m-%dT%H:%M:%S", errors='coerce').dt.tz_localize(None)
parsed_admitted_to_shelter = parsed_admitted_to_shelter.fillna(pd.to_datetime(admitted_to_shelter, errors="coerce").dt.tz_localize(None))
df['DateTime Admitted to Shelter'] = parsed_admitted_to_shelter
print(df['DateTime Admitted to Shelter'].isna().mean())
df = df.drop('Date of Birth', axis=1).drop('DateTime', axis=1)

0.0


In [14]:
import re

def parse_duration(s: str) -> int:
    s = s.strip().lower()
    match = re.fullmatch(r"(\d+)\s*(day|days|week|weeks|month|months|year|years)", s)
    if not match:
        raise ValueError(f"Invalid duration format: {s}")
    unit_to_days = {
        "day": 1, "days": 1,
        "week": 7, "weeks": 7,
        "month": 30, "months": 30,
        "year": 365, "years": 365,
    }
    return int(match.group(1)) * unit_to_days[match.group(2)]
df['Age (days) upon Outcome'] = df['Age upon Outcome'].apply(parse_duration)
print(df['Age (days) upon Outcome'].isna().mean())
df = df.drop('Age upon Outcome', axis=1)

0.0


In [15]:
df['Weekday Entered Shelter'] = df['DateTime Admitted to Shelter'].dt.weekday
df['Month Entered Shelter'] = df['DateTime Admitted to Shelter'].dt.month
df = df.drop('DateTime Admitted to Shelter', axis=1)

In [16]:
df['Color'] = df['Color'].astype(str).str.strip().str.lower().str.replace(r'\s+', ' ', regex=True)
print(df['Color'].apply(lambda x: 'white' in x).mean())
print(df['Color'].apply(lambda x: 'cream' in x).mean())
print(df['Color'].apply(lambda x: 'silver' in x).mean())

0.4449571723859138
0.027191530581891128
0.0030661978384067977


In [17]:
known_colors = ['brown', 'fawn', 'flame', 'lilac', 'seal', 'lynx', 'buff', 'apricot', 'white', 'calico', 'tan', 'black', 'cream', 'orange', 'gray', 'gold', 'red', 'yellow', 'blue', 'chocolate', 'pink', 'tricolor', 'silver', 'green']
known_patterns = ['tabby', 'brindle', 'tiger', 'point', 'smoke', 'tick', 'merle', 'sable_burmese', 'point_siamese', 'torbie', 'tortie']
def coerce_to_slashes(x: str) -> str:
    if x.count('/') == 3:
        return x
    if 'liver' in x:
        return x
    if x in known_colors:
        return f'{x}/{x}/none/none'
    if x in known_patterns:
        return f'none/none/{x}/{x}'
    adjusted = x.replace('sable', 'sable_burmese').replace('point', 'point_siamese').replace('tricolor', 'calico')
    hardcoded_outputs = {
        'blue/cream/white': 'blue/white/none/none',
        'lynx/point_siamese/silver/white': 'lynx/white/none/none',
    }
    if ' ' in x or '/' in adjusted:
        parts = re.split(r'[ /]', adjusted)
        parts.sort()
        parts_ordered = '/'.join(parts)
        if parts_ordered in hardcoded_outputs:
            return hardcoded_outputs[parts_ordered]
        colors = []
        patterns = []
        could_parse=True
        for part in parts:
            if part in known_colors:
                if part not in colors:
                    colors.append(part)
            elif part in known_patterns:
                if part not in patterns:
                    patterns.append(part)
            else:
                print(f'could not parse {x}')
                print(f'failed at part {part}')
                could_parse=False
                break
        if could_parse:
            if len(colors) == 0:
                colors = ['none', 'none']
            elif len(colors) == 1:
                colors.append(colors[0])
            elif len(colors) > 2:
                print(f'too many colors: {colors}')
                return x
            if len(patterns) == 0:
                patterns = ['none', 'none']
            elif len(patterns) == 1:
                patterns.append(patterns[0])
            elif len(patterns) > 2:
                print(f'too many patterns: {patterns}')
                return x
            return '/'.join(colors + patterns)
    return x
df['Color'] = df['Color'].apply(coerce_to_slashes)
print(df['Color'].apply(lambda x: x.count('/') == 3).mean())
df = df[df['Color'].apply(lambda x: x.count('/') == 3)]
df['ColorA'] = df['Color'].apply(lambda x: x.split('/')[0])
df['ColorB'] = df['Color'].apply(lambda x: x.split('/')[1])
df['PatternA'] = df['Color'].apply(lambda x: x.split('/')[2])
df['PatternB'] = df['Color'].apply(lambda x: x.split('/')[3])

too many colors: ['blue', 'brown', 'cream']
too many colors: ['blue', 'buff', 'cream']
too many colors: ['blue', 'calico', 'cream']
could not parse ruddy/cream
failed at part ruddy
could not parse agouti/brown tabby
failed at part agouti
too many patterns: ['point_siamese', 'tabby', 'tortie']
could not parse agouti/white
failed at part agouti
too many colors: ['blue', 'calico', 'cream']
too many colors: ['gray', 'lynx', 'silver']
too many colors: ['blue', 'calico', 'cream']
could not parse brown tabby/agouti
failed at part agouti
could not parse agouti/cream
failed at part agouti
could not parse white/agouti
failed at part agouti
could not parse brown tabby/agouti
failed at part agouti
could not parse brown tabby/agouti
failed at part agouti
could not parse agouti/white
failed at part agouti
could not parse agouti/white
failed at part agouti
could not parse agouti/brown tabby
failed at part agouti
could not parse agouti/gold
failed at part agouti
0.9950422174254616


In [18]:
import numpy as np

df['Weekday_Entered_Shelter_sin'] = np.sin(2 * np.pi * df['Weekday Entered Shelter'] / 7)
df['Weekday_Entered_Shelter__cos'] = np.cos(2 * np.pi * df['Weekday Entered Shelter'] / 7)
df['Month_Entered_Shelter_sin'] = np.sin(2 * np.pi * df['Month Entered Shelter'] / 12)
df['Month_Entered_Shelter_sin'] = np.cos(2 * np.pi * df['Month Entered Shelter'] / 12)
df = df.drop('Color', axis=1).drop('Weekday Entered Shelter', axis=1).drop('Month Entered Shelter', axis=1)
df.head()

,Animal ID,Outcome Type,Animal Type,Sex upon Outcome,Age (days) upon Outcome,ColorA,ColorB,PatternA,PatternB,Weekday_Entered_Shelter_sin,Weekday_Entered_Shelter__cos,Month_Entered_Shelter_sin
0,A680855,0,Bird,Unknown,14,black,yellow,none,none,0.781831,0.62349,-1.0
1,A680857,0,Bird,Unknown,14,black,yellow,none,none,0.781831,0.62349,-1.0
2,A680858,0,Bird,Unknown,14,black,yellow,none,none,0.781831,0.62349,-1.0
3,A680859,0,Bird,Unknown,14,black,yellow,none,none,0.781831,0.62349,-1.0
4,A680860,0,Bird,Unknown,14,black,yellow,none,none,0.781831,0.62349,-1.0


In [21]:
onehot_encoded = pd.get_dummies(df, columns=['Animal Type', 'Sex upon Outcome', 'ColorA', 'ColorB', 'PatternA', 'PatternB'])

In [22]:
x = onehot_encoded.drop(columns=["Outcome Type", "Animal ID"])
y = onehot_encoded["Outcome Type"]

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
num_cols = ['Age (days) upon Outcome']
x_train[num_cols] = scaler.fit_transform(x_train[num_cols])
x_test[num_cols] = scaler.transform(x_test[num_cols])

In [24]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)
y_pred_knn = knn.predict(x_test)

In [25]:
from sklearn.metrics import classification_report
print("KNN")
print(classification_report(y_test, y_pred_knn))
print('-' * 100)

KNN
              precision    recall  f1-score   support

           0       0.83      0.69      0.76      9463
           1       0.84      0.92      0.88     16629

    accuracy                           0.84     26092
   macro avg       0.84      0.81      0.82     26092
weighted avg       0.84      0.84      0.83     26092

----------------------------------------------------------------------------------------------------


In [26]:
from sklearn.model_selection import GridSearchCV

param_grid = {"n_neighbors": [3, 5, 7, 9, 11]}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring="f1")
grid.fit(x_train, y_train)
y_pred_knn_grid = grid.predict(x_test)

In [27]:
print("KNN (GridSearchCV)")
print(classification_report(y_test, y_pred_knn_grid))
print('-' * 100)

KNN (GridSearchCV)
              precision    recall  f1-score   support

           0       0.88      0.68      0.76      9463
           1       0.84      0.95      0.89     16629

    accuracy                           0.85     26092
   macro avg       0.86      0.81      0.83     26092
weighted avg       0.85      0.85      0.84     26092

----------------------------------------------------------------------------------------------------


In [28]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(max_iter=1000)
logreg.fit(x_train, y_train)
y_pred_logreg = logreg.predict(x_test)

In [29]:
print("Logistic Regression:")
print(classification_report(y_test, y_pred_logreg))

Logistic Regression:
              precision    recall  f1-score   support

           0       0.90      0.68      0.78      9463
           1       0.84      0.96      0.90     16629

    accuracy                           0.86     26092
   macro avg       0.87      0.82      0.84     26092
weighted avg       0.86      0.86      0.85     26092



# Use of AI
[1]. 
    Tool: ChatGPT

    Prompt: "python pandas df how can I check if there are any rows where 'MonthYear' is present but 'DateTime' is not"

    Output:
        # Boolean mask
        mask = df['MonthYear'].notna() & df['DateTime'].isna()
        
        # Check if any rows match
        has_missing = mask.any()
        print(has_missing)
[2].
    Tool: ChatGPT

    Prompt: "how to count what portion of values in 'Outcome Subtype' that are NOT non-empty strings ie they're not a string or they're nan or they're empty strings or whatever Also how to count what portion of values in 'Sex upon Outcome' are 'Unknown'"

    Output:
        mask = ~df['Outcome Subtype'].apply(lambda x: isinstance(x, str) and x.strip() != "")

        # Count how many
        num_invalid = mask.sum()
        
        # Portion (fraction of total rows)
        portion_invalid = num_invalid / len(df)
        
        print(f"Invalid Outcome Subtype: {num_invalid} rows, {portion_invalid:.2%} of total")

        num_unknown = (df['Sex upon Outcome'] == 'Unknown').sum()
        portion_unknown = num_unknown / len(df)
        
        print(f"Unknown Sex upon Outcome: {num_unknown} rows, {portion_unknown:.2%} of total")
[3].
    Tool: ChatGPT

    Prompt: "python parse mm/dd/yy string into datetime and also parse strings like 2014-06-10T00:00:00-05:00 into datetime Also, the mm in the first one might not have leading zeros"

    Output:
        import pandas as pd

        dates = [
            "6/3/14",               # mm/dd/yy without leading zeros
            "12/25/20",             # mm/dd/yy with leading zeros
            "2014-06-10T00:00:00-05:00"  # ISO 8601 with timezone
        ]
        
        # Parse into datetime
        parsed = pd.to_datetime(dates, errors="coerce")
        
        print(parsed)
[4].
    Tool: ChatGPT
    
    Prompt: "write a python parse function that takes inputs like 'n days/weeks/months/years' where n can be any real nonnegative integer and days/weeks/months/years may or may not have the trailing s and converts it to a number of days"

    Output:
        import re

        def parse_duration(s: str) -> int:
            """
            Parse strings like 'n day(s)/week(s)/month(s)/year(s)' into number of days.
            n must be a nonnegative integer.
            
            Assumptions:
              - 1 day = 1 day
              - 1 week = 7 days
              - 1 month = 30 days (approx)
              - 1 year = 365 days (approx)
            """
            if not isinstance(s, str):
                raise ValueError("Input must be a string")
        
            # Normalize whitespace and lowercase
            s = s.strip().lower()
        
            # Regex: capture number and unit
            match = re.fullmatch(r"(\d+)\s*(day|days|week|weeks|month|months|year|years)", s)
            if not match:
                raise ValueError(f"Invalid duration format: {s}")
        
            n = int(match.group(1))
            unit = match.group(2)
        
            # Conversion map
            unit_to_days = {
                "day": 1, "days": 1,
                "week": 7, "weeks": 7,
                "month": 30, "months": 30,
                "year": 365, "years": 365,
            }
        
            return n * unit_to_days[unit]
[5].
    Tool: ChatGPT
    Prompt: "how to make a onehot encoding from a sorted list of categorical vars and then onehot encode categorical columns in a df (they are already sanitized strings)"
    Output:
        import pandas as pd

        cats = ["dog", "cat", "dog", "bird", "cat"]
        
        # Sorted unique categories
        unique_cats = sorted(set(cats))
        
        # One-hot encode
        onehot = pd.get_dummies(cats, columns=None)
        print(onehot)
[6].
    Tool: ChatGPT
    Prompt: should I make weekday and month onehot vars?
    Output:
        (It yapped about onehot vs numeric but I got real interested when it suggested cyclical encoding)
        import numpy as np

        # Weekday 0=Mon, 6=Sun
        df["Weekday_sin"] = np.sin(2 * np.pi * df["Weekday Entered Shelter"] / 7)
        df["Weekday_cos"] = np.cos(2 * np.pi * df["Weekday Entered Shelter"] / 7)
        
        # Month 1-12
        df["Month_sin"] = np.sin(2 * np.pi * (df["Month Entered Shelter"]-1) / 12)
        df["Month_cos"] = np.cos(2 * np.pi * (df["Month Entered Shelter"]-1) / 12)